Outliers

Outliers affect the model in a multiplicity of ways: instability (new data would produce completely different model, because the current one is very relying on its outliers), biasness (the noise capture would over-influence parameters' estimation by leading to altered values), overfitting & generalization (train vs. test error)

In [43]:
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix

k = 3

# Sample dataset of 10 points in 2D space
data = pd.DataFrame({'point': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'x': [1, 2, 1, 5, 6, 8, 3, 2, 7, 9],
        'y': [1, 1, 2, 5, 5, 8, 3, 2, 6, 9]}).set_index(['point'])

points = np.array(list(zip(data['x'], data['y'])))
dist_matrix = pd.DataFrame(distance_matrix(points, points))
display(data.T)
display(dist_matrix)

point,1,2,3,4,5,6,7,8,9,10
x,1,2,1,5,6,8,3,2,7,9
y,1,1,2,5,5,8,3,2,6,9


,0,1,2,3,4,5,6,7,8,9
0,0.000000,1.000000,1.000000,5.656854,6.403124,9.899495,2.828427,1.414214,7.810250,11.313708
1,1.000000,0.000000,1.414214,5.000000,5.656854,9.219544,2.236068,1.000000,7.071068,10.630146
2,1.000000,1.414214,0.000000,5.000000,5.830952,9.219544,2.236068,1.000000,7.211103,10.630146
3,5.656854,5.000000,5.000000,0.000000,1.000000,4.242641,2.828427,4.242641,2.236068,5.656854
4,6.403124,5.656854,5.830952,1.000000,0.000000,3.605551,3.605551,5.000000,1.414214,5.000000
5,9.899495,9.219544,9.219544,4.242641,3.605551,0.000000,7.071068,8.485281,2.236068,1.414214
6,2.828427,2.236068,2.236068,2.828427,3.605551,7.071068,0.000000,1.414214,5.000000,8.485281
7,1.414214,1.000000,1.000000,4.242641,5.000000,8.485281,1.414214,0.000000,6.403124,9.899495
8,7.810250,7.071068,7.211103,2.236068,1.414214,2.236068,5.000000,6.403124,0.000000,3.605551
9,11.313708,10.630146,10.630146,5.656854,5.000000,1.414214,8.485281,9.899495,3.605551,0.000000


In [42]:
k_distances, neighborhoods = [], []
for i in range(dist_matrix.shape[0]): # for each point...
    distances = np.sort(dist_matrix[i]) # get the ordered i-th row (all distances from a given point)
    k_boundary = distances[k] # define the distance with the k-th point from the selected one
    neighborhood = np.where(dist_matrix[i] <= k_boundary)[0] # collect all the points with a distance below the boundary
    neighborhoods.append(neighborhood), k_distances.append(k_boundary) # save neighbors points and k-th distance (one distance per row)
display(dist_matrix)

,0,1,2,3,4,5,6,7,8,9
0,0.000000,1.000000,1.000000,5.656854,6.403124,9.899495,2.828427,1.414214,7.810250,11.313708
1,1.000000,0.000000,1.414214,5.000000,5.656854,9.219544,2.236068,1.000000,7.071068,10.630146
2,1.000000,1.414214,0.000000,5.000000,5.830952,9.219544,2.236068,1.000000,7.211103,10.630146
3,5.656854,5.000000,5.000000,0.000000,1.000000,4.242641,2.828427,4.242641,2.236068,5.656854
4,6.403124,5.656854,5.830952,1.000000,0.000000,3.605551,3.605551,5.000000,1.414214,5.000000
5,9.899495,9.219544,9.219544,4.242641,3.605551,0.000000,7.071068,8.485281,2.236068,1.414214
6,2.828427,2.236068,2.236068,2.828427,3.605551,7.071068,0.000000,1.414214,5.000000,8.485281
7,1.414214,1.000000,1.000000,4.242641,5.000000,8.485281,1.414214,0.000000,6.403124,9.899495
8,7.810250,7.071068,7.211103,2.236068,1.414214,2.236068,5.000000,6.403124,0.000000,3.605551
9,11.313708,10.630146,10.630146,5.656854,5.000000,1.414214,8.485281,9.899495,3.605551,0.000000


In [49]:
reach_dist = pd.DataFrame(np.zeros_like(dist_matrix))
for i in range(dist_matrix.shape[0]): # for each point ...
    for j in range(dist_matrix.shape[1]): # for each other point ...
        if j in neighborhoods[i]: # if it's a neighbor
            reach_dist.iloc[i, j] = max(k_distances[j], dist_matrix.values[i, j]) # ... max(neighbor's k-distance, actual distance)
display(reach_dist)

,0,1,2,3,4,5,6,7,8,9
0,1.414214,1.414214,1.414214,0.000000,0.000000,0.000000,0.000000,1.414214,0.000000,0.0
1,1.414214,1.414214,1.414214,0.000000,0.000000,0.000000,0.000000,1.414214,0.000000,0.0
2,1.414214,1.414214,1.414214,0.000000,0.000000,0.000000,0.000000,1.414214,0.000000,0.0
3,0.000000,0.000000,0.000000,2.828427,3.605551,0.000000,2.828427,0.000000,2.236068,0.0
4,0.000000,0.000000,0.000000,2.828427,3.605551,3.605551,3.605551,0.000000,2.236068,0.0
5,0.000000,0.000000,0.000000,0.000000,3.605551,3.605551,0.000000,0.000000,2.236068,5.0
6,0.000000,2.236068,2.236068,0.000000,0.000000,0.000000,2.236068,1.414214,0.000000,0.0
7,1.414214,1.414214,1.414214,0.000000,0.000000,0.000000,2.236068,1.414214,0.000000,0.0
8,0.000000,0.000000,0.000000,2.828427,3.605551,3.605551,0.000000,0.000000,2.236068,0.0
9,0.000000,0.000000,0.000000,0.000000,5.000000,3.605551,0.000000,0.000000,3.605551,5.0


In [52]:
lrd = []
for i in range(reach_dist.shape[0]): # for each point ...
    a = len(neighborhoods[i]) # n° of points inside the boundary
    b = np.sum(reach_dist.iloc[i, neighborhoods[i]].values) # sum of reachability distances
    fact = a / b if b != 0 else 0 # local reachability density
    lrd.append(fact)
display(pd.Series(lrd))

0    0.707107
1    0.707107
2    0.707107
3    0.347872
4    0.314839
5    0.276871
6    0.492464
7    0.633479
8    0.325850
9    0.232408
dtype: float64

In [55]:
lof = []
for i in range(reach_dist.shape[0]): # for each point ...
    print(lrd[neighborhoods[i][0]])
    a = np.sum(lrd[neigh] / lrd[i] for neigh in neighborhoods[i]) # for each point within the boundary take the ratio lrd(neighbor)/lrd(selected point)
    b = len(neighborhoods[i]) # n° points within the k-th distance boundary
    c = a / b if b != 0 else 0 # average of these ratios
    print()
    lof.append(c)
display(pd.Series(lof))

0.7071067811865475

0.7071067811865475

0.7071067811865475

0.3478722631439356

0.3478722631439356

0.31483868216761485

0.7071067811865475

0.7071067811865475

0.3478722631439356

0.31483868216761485



C:\Users\matte\AppData\Local\Temp\ipykernel_3804\907001065.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  a = np.sum(lrd[neigh] / lrd[i] for neigh in neighborhoods[i]) # for each point within the boundary take the ratio lrd(neighbor)/lrd(selected point)


0    0.973969
1    0.973969
2    0.973969
3    1.064345
4    1.116696
5    1.038361
6    1.289513
7    1.025216
8    0.970870
9    1.237013
dtype: float64